In [2]:
import cirq

In [5]:
import random

num_bits = 50 # A random choice for the length of secret key for encryption 
alice_key = random.choices([0, 1], k = num_bits) # Alice pick up the Secret Key 
alice_bases = random.choices(['Z', 'X'], k = num_bits) # Alice Picks the Basis

qubits = cirq.NamedQubit.range(num_bits, prefix = 'q') # Alice creates qubits to store the bit values

basis_gates = {0:cirq.I , 1: cirq.X} # Encoding the binary key bit value into qubit. Binary 0 is encoded as Qubit state 0 and Binary 1 is encoded as qubit state 1.
encode_basis = {'Z':cirq.I, 'X':cirq.H} # Encoding basis into the prepared qubit as per Alice random basis choice for each qubit.

# ALICE CREATES HER CIRCUIT TO ENCODE THE RANDOMLY CHOSEN BASIS TO THE QUBIT

alice_circuit = cirq.Circuit()

for bit in range(num_bits):

  encode_qubit_value = alice_key[bit] #Alice choses each bit in secret key and stores it in a variable
  encode_gate = basis_gates[encode_qubit_value] # She encodes the bit state into a qubit according to the defined criteria.

  basis_value=alice_bases[bit] # Alice choses the randomly created basis states for encoding it into her qubit.
  basis_gate = encode_basis[basis_value] # Alice encodes the basis state into a qubit according to the defined criteria.

  qubit = qubits[bit] # Alice choses one qubit by each
  alice_circuit.append(encode_gate(qubit)) # Encodes the corrsponding state for 0 and 1 into it.
  alice_circuit.append(basis_gate(qubit)) # Encodes the basis state 

# BOB CREATES HIS CIRCUIT TO RNADOMLY PICK BASES AND MEASEURE THE RECIEVED QUBIT


bob_encode_basis ={'Z':cirq.I ,'X' : cirq.H}  # Encoding basis into the recieved qubits.
bob_basis = random.choices(['Z', 'X'], k = num_bits) # BOB Choses random basis and append it to the recieved qubits form alice

bob_circuit = cirq.Circuit()

for bit in range (num_bits):
  basis_value = bob_basis[bit]
  basis_gate = bob_encode_basis[basis_value] # BOB encodes the basis state into the qubits according to the defined criteria.

  qubit = qubits[bit]
  bob_circuit.append(basis_gate(qubit)) # Encodes the basis state 

# BOB MEASURES HIS CIRCUIT 

bob_circuit.append(cirq.measure(qubits , key = "bob key"))

# BOB RE-CREATES THE SECRET KEY

bb84_circuit = alice_circuit + bob_circuit

sim = cirq.Simulator()
results = sim.run(bb84_circuit)
bob_key = results.measurements['bob key'][0]

#ALICE AND BOB COMPARE BASES

final_alice_key = []
final_bob_key = []

for bit in range(num_bits):

  if alice_bases[bit] == bob_basis[bit]:
    final_alice_key.append(alice_key[bit])
    final_bob_key.append(bob_key[bit])

print('\nAlice\'s key: ', final_alice_key)
print('Bob\'s key: ', final_bob_key)

#ALICE AND BOB COMPARES THE FIRST FEW BITS INORDER TO DETECT THE PRESCENCE OF EVE

if final_alice_key[0] == final_bob_key[0]:
  final_alice_key = final_alice_key[1:]
  final_bob_key = final_bob_key[1:]

  print('We can use our keys!')
  print('Alice Key: ', final_alice_key)
  print('Bob Key: ', final_bob_key)


Alice's key:  [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]
Bob's key:  [1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]
We can use our keys!
Alice Key:  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]
Bob Key:  [1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0]


In [8]:
secret_key = final_alice_key[0:8]
secret_key

[1, 0, 0, 1, 0, 0, 0, 1, 1]